# 手写识别

In [319]:
import numpy as np
import pandas as pd
#from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

In [320]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

In [321]:
train_data.head()
train_data.shape

(42000, 785)

In [322]:
dummies = pd.get_dummies(train_data['label'], prefix='label', drop_first=False)
train_data_oh = train_data
train_data_oh = train_data_oh.drop('label', axis=1)
train_data_oh = pd.concat([dummies, train_data_oh], axis=1)
train_data_oh.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [323]:
test_data.shape
# dummies = pd.get_dummies(test_data['label'], prefix='label', drop_first=False)
# test_data_oh = test_data
# test_data_oh = test_data_oh.drop('label', axis=1)
# test_data_oh = pd.concat([dummies, test_data_oh], axis=1)
# test_data_oh.head()

(28000, 784)

In [324]:
in_units = 784
w = tf.Variable(tf.truncated_normal([in_units, 10], stddev=0.1), name='weight')
b = tf.Variable(tf.zeros([10]))

In [325]:
x = tf.placeholder(tf.float32, [None, in_units])
y = tf.nn.softmax(tf.matmul(x, w) + b)

In [326]:
y_ = tf.placeholder(tf.float32, [None, 10])
res = tf.one_hot(labels, 10)

In [327]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [328]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [329]:
with tf.Session() as train:
    tf.global_variables_initializer().run()
    for i in range(1000):
        #print("{0} epoc start\n".format(i))
        random_batch = train_data_oh.loc[np.random.randint(42000, size=100)]
        images = random_batch.iloc[:, 10:].values
        images = images.astype(np.float)
        # convert from [0:255] => [0.0:1.0]
        images = np.multiply(images, 1.0 / 255.0)
        
        labels = random_batch.iloc[:, 0:10].values
        
        #print("{0} epoc end".format(i))
        train_step.run({x: images, y_: labels})
    
    #shape = predictions.shape
    predictions = train.run([y], feed_dict={x: test_data})
    predictions = np.array(predictions)
    shape = predictions.shape
    predictions= np.reshape(predictions, (shape[0] * shape[1], shape[2]))
    print(predictions.shape)
    #predictions = np.array(predictions)
    #print(type(predictions))
    #print(np.array(predictions))
    #print(np.argmax(predictions, 1))
    y_pre = np.argmax(predictions, 1)
    conv_y_submission = pd.DataFrame({
        'ImageId': range(1, 28001), 
        'Label': y_pre 
    })

conv_y_submission.head()
print(conv_y_submission)
conv_y_submission.to_csv('./conv_submission.csv', index=False)
print("训练完成")

(28000, 10)
       ImageId  Label
0            1      2
1            2      0
2            3      9
3            4      9
4            5      3
5            6      7
6            7      0
7            8      3
8            9      0
9           10      3
10          11      5
11          12      9
12          13      4
13          14      0
14          15      4
15          16      0
16          17      3
17          18      1
18          19      9
19          20      0
20          21      9
21          22      1
22          23      8
23          24      5
24          25      7
25          26      4
26          27      2
27          28      7
28          29      8
29          30      7
...        ...    ...
27970    27971      5
27971    27972      0
27972    27973      4
27973    27974      8
27974    27975      0
27975    27976      3
27976    27977      6
27977    27978      0
27978    27979      1
27979    27980      9
27980    27981      3
27981    27982      1
27982    27983      